---

# Pre-processing
## Gộp các dữ liệu thu thập được vào một file duy nhất, loại bỏ các dòng chứa giá trị NULL và xóa các dữ liệu bị lặp

In [ ]:
import pandas as pd
import os

def clean_and_merge_csv_files(input_folder, file_names, output_folder, output_file, chunk_size=50000):
    os.makedirs(output_folder, exist_ok=True)

    merged_df = pd.DataFrame() 

    for file_name in file_names:
        file_path = os.path.join(input_folder, file_name)
        print(f"📂 Đang xử lý file: {file_path}")

        # Đọc file theo chunksize
        for chunk in pd.read_csv(file_path, chunksize=chunk_size, dtype=str):
            # Chuẩn hóa tên cột (chuyển về chữ thường, loại bỏ khoảng trắng)
            chunk.columns = [col.lower().strip() for col in chunk.columns]

            # 🛑 Xóa toàn bộ dòng có giá trị NULL ở bất kỳ cột nào
            chunk_clean = chunk.dropna()

            # Gộp dữ liệu vào DataFrame chính
            merged_df = pd.concat([merged_df, chunk_clean], ignore_index=True)

    # Xóa trùng lặp dựa trên cột 'title'
    merged_df.drop_duplicates(subset=['title'], inplace=True)

    # Lưu kết quả
    output_path = os.path.join(output_folder, output_file)
    merged_df.to_csv(output_path, index=False)

    print(f"\n✅ Tổng số dòng sau khi làm sạch và gộp: {merged_df.shape[0]}")
    print(f"📂 Dữ liệu đã được lưu tại: {output_path}")

# Thư mục chứa tệp đầu vào và tên các tệp cần xử lý
input_folder = "dataraw"
files = ["data_thanhnien.csv", "tuoitretotal.csv", "vnexpress_articles.csv"]

# Thư mục và tệp đầu ra
output_folder = "data_process_1"
output_file = "data_no_null.csv"

# Gọi hàm
clean_and_merge_csv_files(input_folder, files, output_folder, output_file)


📂 Đang xử lý file: dataraw\data_thanhnien.csv
📂 Đang xử lý file: dataraw\tuoitretotal.csv
📂 Đang xử lý file: dataraw\vnexpress_articles.csv

✅ Tổng số dòng sau khi làm sạch và gộp: 1339798
📂 Dữ liệu đã được lưu tại: data_process_1\data_no_null.csv


## Kiểm tra lại data

In [12]:
import pandas as pd

file_path = "data_process_1/data_no_null.csv"
chunk_size = 50000  

null_counts = None

for chunk in pd.read_csv(file_path, chunksize=chunk_size):
    if null_counts is None:
        null_counts = chunk.isnull().sum()  
    else:
        null_counts += chunk.isnull().sum()  

print("\n🛑 Số giá trị NULL trên từng cột trong toàn bộ dữ liệu:")
print(null_counts)



🛑 Số giá trị NULL trên từng cột trong toàn bộ dữ liệu:
url               0
title             0
abstract          0
content           0
category      38688
genre       1301110
dtype: int64


## Loại bỏ cột genre, xóa các dòng có chứa giá trị cột category trống

In [ ]:
import pandas as pd
import os

# Đường dẫn file gốc
file_path = "data_process_1/data_no_null.csv"
output_folder = "data_process_2"
output_file = "data_no_genre_and_category_null.csv"
chunk_size = 50000  # Xử lý từng phần nhỏ

# Tạo thư mục đầu ra
os.makedirs(output_folder, exist_ok=True)
output_path = os.path.join(output_folder, output_file)

# Xử lý dữ liệu theo chunks
df_clean = pd.DataFrame() 

with pd.read_csv(file_path, chunksize=chunk_size) as reader:
    for chunk in reader:
        chunk_clean = chunk.dropna(subset=['category'])
        chunk_clean = chunk_clean.drop(columns=['genre'])
        # Gộp dữ liệu 
        df_clean = pd.concat([df_clean, chunk_clean], ignore_index=True)

# Lưu kết quả vào file mới trong thư mục data_process_2
df_clean.to_csv(output_path, index=False)

print(f"\n✅ Dữ liệu đã được làm sạch và lưu lại tại: {output_path}")



✅ Dữ liệu đã được làm sạch và lưu lại tại: data_process_2\data_no_genre_and_category_null.csv


## Kiểm tra lại dữ liệu

In [ ]:
import pandas as pd

file_path = "data_process_2/data_no_genre_and_category_null.csv"
chunk_size = 50000  

null_counts = None
title_duplicates = 0  # Biến đếm số dòng trùng lặp theo cột 'title'

for chunk in pd.read_csv(file_path, chunksize=chunk_size):
    # Kiểm tra giá trị NULL
    if null_counts is None:
        null_counts = chunk.isnull().sum()
    else:
        null_counts += chunk.isnull().sum()

    # Kiểm tra các dòng trùng lặp theo cột 'title'
    title_duplicates += chunk.duplicated(subset=['title']).sum()

print("\n🛑 Số giá trị NULL trên từng cột trong toàn bộ dữ liệu:")
print(null_counts)

print(f"\n📝 Số dòng trùng lặp theo cột 'title': {title_duplicates}")



🛑 Số giá trị NULL trên từng cột trong toàn bộ dữ liệu:
url         0
title       0
abstract    0
content     0
category    0
dtype: int64

📝 Số dòng trùng lặp theo cột 'title': 0


## Chuyển cột category thành chữ thường và kiểm tra số thể loại được đề cập

In [2]:
import pandas as pd
import os
# Đường dẫn đến file dữ liệu
file_path = "data_process_2/data_no_genre_and_category_null.csv"
output_folder = "data_process_3"
output_file = "data_no_genre_and_category_lower.csv"

chunk_size = 50000

os.makedirs(output_folder, exist_ok=True)
output_path = os.path.join(output_folder, output_file)

# Lưu số lượng dòng theo từng loại category
category_counts = {}

# Đọc tệp dữ liệu theo từng chunk
for chunk in pd.read_csv(file_path, chunksize=chunk_size):
    # Chuyển cột 'category' về chữ thường
    chunk['category'] = chunk['category'].str.lower()
    
    # Kiểm tra số dòng của mỗi loại category
    category_counts_chunk = chunk['category'].value_counts()
    
    # Cộng dồn vào số lượng category đã có
    category_counts.update(category_counts_chunk.to_dict())

    # Ghi dữ liệu sau khi đã chuyển cột category về chữ thường vào file mới
    chunk.to_csv(output_path, mode='a', header=not os.path.exists(output_path), index=False)

# In số lượng dòng của từng loại category
print("\n📊 Số lượng dòng của từng loại 'category':")
for category, count in category_counts.items():
    print(f"'{category}': {count}")

print(f"\n✅ Dữ liệu đã được xử lý và lưu tại: {output_path}")



📊 Số lượng dòng của từng loại 'category':
'thời sự': 57
'công nghệ': 966
'thể thao': 142
'thế giới': 90
'văn hóa': 68
'kinh tế': 2228
'video': 2452
'giải trí': 1
'sức khỏe': 46
'giáo dục': 22
'đời sống': 1222
'giới trẻ': 822
'tno': 4
'xe': 12
'du lịch': 12
'tiêu dùng thông minh': 365
'chính trị': 708
'bạn cần biết': 302
'bạn đọc': 198
'chào ngày mới': 81
'blog phóng viên': 41
'podcast': 2
'tôi viết': 11
'diễn đàn': 43
'nhật ký tết việt': 27
'kinh doanh': 84
'pháp luật': 44
'bạn đọc làm báo': 44
'cần biết': 12
'nhịp sống trẻ': 67
'kinh doanh, doanh nghiệp': 148
'nhà đất': 19
'thời sự, thời tiết': 1
'khoa học': 11
'thời sự, phóng sự': 21
'giả - thật': 222
'thời sự, bình luận': 10
'kinh doanh, tài chính': 13
'sức khỏe, biết để khỏe': 1000
'kinh doanh, mua sắm': 26
'thời sự, xã hội': 201
'thời sự, bút bi': 144
'kinh doanh, đầu tư': 42
'nhịp sống trẻ, tuổi trẻ start-up award': 22
'nhà đất, dự án': 3
'giáo dục, nhịp sống học đường': 200
'công nghệ, nhịp sống số': 25
'giáo dục, tuyển sinh': 

In [7]:
import nltk

# Tải lại tài nguyên 'punkt'
nltk.download('punkt', force=True)


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\PhatK\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True